In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from lxml import etree
import requests
import time
import re
import os
from ctypes import *

In [74]:
class PictureUrl:
    def __init__(self, url_info, host_headers):
        picture_url = url_info.split(';')[0]
        self.title = url_info.split(';')[2]
        self.start_number = url_info.split(';')[1].split('-')[0]
        self.end_number = url_info.split(';')[1].split('-')[1]
        self.save_path = "d:/pictures/"
        self.urls = []
        self.href = picture_url
        self.is_found = True

        self.host_headers = host_headers
        self.item_number = re.findall('(\d+)', self.href)[0]
        file_path = self.save_path + re.sub('[\/:*?"<>|]', '', self.title.strip())
        if not os.path.exists(file_path):
            os.makedirs(file_path)
        self.file_path = file_path

    def get_image_url(self):
        if self.is_found:
            html = requests.get(self.href, headers=self.host_headers)
            selector = etree.HTML(html.text)
            image_prefix = selector.xpath('.//ul[@id="hgallery"]/img/@src')[0]
            index = image_prefix.rfind(r'/')
            image_prefix = image_prefix[0:index+1]
            self.urls.append(image_prefix + "0.jpg")
            for j in range(int(self.start_number), int(self.end_number)):
                index = self.href.find(".html")
                url = image_prefix + str(j).zfill(3) + ".jpg"
                self.urls.append(url)

In [75]:
class CrawlerPictures:
    def __init__(self, urls_file):
        self.urls_file = urls_file
        self.max_page_number = 1
        self.first_layer_urls = []
        self.picture_urls = []
        self.host_headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/67.0.3396.87 Safari/537.36',
        }

        self.picture_headers = {
            'Sec-Fetch-Dest': 'image',
            'Host':'t1.onvshen.com:85',
            'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
            'Accept-Encoding': 'gzip, deflate, br',
            'Accept-Language': 'zh-CN,zh;q=0.9',
            'Cache-Control': 'max-age=0',
            'Connection': 'keep-alive',
            'Sec-Fetch-Mode': 'navigate',
            'Sec-Fetch-Site': 'same-origin',
            'Sec-Fetch-User': '?1',
            'Upgrade-Insecure-Requests': '1',
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/67.0.3396.87 Safari/537.36',
        }

    # 获取第一层所有url
    def get_first_layer_url(self):
        with open(self.urls_file, 'r', encoding='UTF-8') as file_lines:
            lines = file_lines.readlines()
            for line in lines:
                picture = PictureUrl(line, self.host_headers)
                picture.get_image_url()
                self.first_layer_urls.append(picture)

    # 获取所有图片
    def get_images(self):
        print("start get images")
        for picture in self.first_layer_urls:
            print("start get " + picture.title)
            for url in picture.urls:
                self.get_image(url, picture.file_path)
            print("end get " + picture.title)
                
    def get_image(self, url, file_path):
        host = url.split(r'/')[2]
        self.picture_headers['Host'] = host
        image_html = requests.get(url, headers=self.picture_headers)
        if image_html.status_code == 200:
            file_name = file_path + '/' + url.split(r'/')[-1]
            with open(file_name, 'wb') as file_obj:
                file_obj.write(image_html.content)
        else:
            print("get image failed: " + url)
        #time.sleep(1)


In [79]:
crawler_picture = CrawlerPictures('urls_nv.txt')
crawler_picture.get_first_layer_url()
crawler_picture.get_images()

start get images
start get [YALAYI雅拉伊] 2019.06.23 Vol.317 魔镜魔镜告诉我 凉儿

end get [YALAYI雅拉伊] 2019.06.23 Vol.317 魔镜魔镜告诉我 凉儿

start get [YALAYI雅拉伊] 2019.06.05 Y299 凉儿《香蜜》

end get [YALAYI雅拉伊] 2019.06.05 Y299 凉儿《香蜜》

start get [YALAYI雅拉伊] 2019.01.01 No.157 童话 凉儿

end get [YALAYI雅拉伊] 2019.01.01 No.157 童话 凉儿

start get [YALAYI]雅拉伊 2018.12.25 No.154 圣诞女皇 凉儿

end get [YALAYI]雅拉伊 2018.12.25 No.154 圣诞女皇 凉儿

start get [YALAYI雅拉伊] 2018.12.24 No.150 橘色的暖阳花 凉儿
end get [YALAYI雅拉伊] 2018.12.24 No.150 橘色的暖阳花 凉儿


In [64]:
url = "https://img.onvshen.com:85/gallery/27598/32453/s/001.jpg"

# User-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.149 Safari/537.36
host_headers = {
    #'Referer': 'https://www.nvshens.net/g/32866/',
    'Sec-Fetch-Dest': 'document',
    'Host':'img.onvshen.com:85',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept-Language': 'zh-CN,zh;q=0.9',
    'Cache-Control': 'max-age=0',
    'Connection': 'keep-alive',
    #'Cookie': 'Hm_lvt_1bb490b9b92efa278bd96f00d3d8ebb4=1590409255; Hm_lvt_1752519ac23aa7d83b98d4602cffe1a2=1590409433; gallery_32866=1; gallery_undefined=1; records=%5B%7B%22id%22%3A%2227598%22%2C%22name%22%3A%22%u9676%u559C%u4E50%22%7D%2C%7B%22id%22%3A%2222204%22%2C%22name%22%3A%22%u5C0F%u70ED%u5DF4%22%7D%5D; gallery_32606=1; Hm_lpvt_1752519ac23aa7d83b98d4602cffe1a2=1590411200; Hm_lpvt_1bb490b9b92efa278bd96f00d3d8ebb4=1590411932',
    'Sec-Fetch-Mode': 'navigate',
    'Sec-Fetch-Site': 'none',
    'Sec-Fetch-User': '?1',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/67.0.3396.87 Safari/537.36',
        }

html = requests.get(url, headers=host_headers)
print(html.status_code)

file_name = r'D:\pictures\1.jpg'
with open(file_name, 'wb') as file_obj:
    file_obj.write(html.content)

200


In [66]:
url = "https://t1.onvshen.com:85/gallery/27598/32866/s/002.jpg"

print(url.split(r'/')[2])
print()

t1.onvshen.com:85



In [50]:
n = 5
n1 = "5"
print(n1.zfill(3))
print(format(n, "05d"))

005
00005
